# with 和 上下文管理器

## 上下文管理器

- 上下文管理器是指在一段代码执行之前执行一段代码，用于一些预处理工作；执行之后再执行一段代码，用于一些清理工作。
- 比如打开文件进行读写，读写完之后需要将文件关闭。又比如在数据库操作中，操作之前需要连接数据库，操作之后需要关闭数据库。
- 在上下文管理协议中，有两个方法`__enter__`和`__exit__`，分别实现上述两个功能。
- 任何实现了 `__enter__()` 和`__exit__()` 方法的对象都可称之为上下文管理器，上下文管理器对象可以使用 with 关键字。显然，文件（file）对象也实现了上下文管理器。

## with语法

    with Expression as Var:
        Block
        
- 这里就是一个标准的上下文管理器的使用逻辑，稍微解释一下其中的运行逻辑：
    1. 执行 `Expression` 语句，获取上下文管理器（Context Manager）
    2. 调用上下文管理器中的`__enter__`方法，该方法执行一些预处理工作。
    3. 这里的 `as Var` 可以省略，如果不省略，则将`__enter__`方法的返回值赋值给 `Var`。
    4. 执行代码块 `Block`，这里的 `Var` 可以当做普通变量使用。
    5. 最后调用上下文管理器中的的`__exit__`方法。
    6. `__exit__`方法有三个参数：exc_type, exc_val, exc_tb。如果代码块 `block` 发生异常并退出，那么分别对应异常的type、value 和 traceback。否则三个参数全为None。
    7. `__exit__`方法的返回值可以为True或者False。如果为True，那么表示异常被忽视，相当于进行了try-except操作；如果为False，则该异常会被重新raise。

In [12]:
# 实现一个简单的上下文管理器
class File(object):
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        
    def __enter__(self):
        """预处理工作"""
        print("enter")
        self.f = open(self.filename, self.mode, encoding="utf-8")
        return self.f

    def __exit__(self, exc_type, exc_val, exc_tb):
        """清理工作"""
        print("exc_type:", exc_type)
        print("exc_val:", exc_val)
        print("exc_tb:", exc_tb)
        self.f.close()
        print("exit")
        

with File('test.txt', 'w') as f:
    f.write("自定义上下文管理器")


enter
exc_type: None
exc_val: None
exc_tb: None
exit


- 因为上面 File 类实现了上下文管理器，现在就可以使用 with 语句了，并且无需显示地调用 close 方法了，由系统自动去调用，哪怕中间遇到异常 close 方法也会被调用。

## 实现上下文管理器的其他方式

- Python 还提供了一个 contextmanager 的装饰器，更进一步简化了上下文管理器的实现方式。通过 yield 将函数分割成两部分，yield 之前的语句在 `__enter__` 方法中执行，yield 之后的语句在 `__exit__` 方法中执行。紧跟在 yield 后面的值是函数的返回值赋值给 as 后的变量。

In [15]:
from contextlib import contextmanager

@contextmanager
def open_file(filename, mode):
    print("预准备")
    f = open(filename, mode, encoding="utf-8")
    yield f
    f.close()
    print("结束")
    

with open_file('test.txt', "a") as f:
    f.write("实现上下文管理器的其他方式")


预准备
结束


## 总结

- Python 提供了 with 语法用于简化资源操作的后续清除操作，是 try/finally 的替代方法，实现原理建立在上下文管理器之上。此外，Python 还提供了一个 contextmanager 装饰器，更进一步简化上下管理器的实现方式。

- [参考](https://zhuanlan.zhihu.com/p/24709718)